In [42]:
import pysal
from pysal.weights.spatial_lag import lag_spatial as slag
from pysal.esda.moran import Moran
import scipy.stats as stats
import scipy.spatial.distance as dist
import numpy as np
w = pysal.open(pysal.examples.get_path("stl.gal")).read()
f = pysal.open(pysal.examples.get_path("stl_hom.txt"))
y = np.array(f.by_col['HR8893'])
mi = Moran(y,  w)
print "%7.5f" % mi.I



0.24366


In [3]:
class BasicMoran:
    """Moran's I Global Autocorrelation Statistic
    Parameters
    ----------
    y               : array
                      variable measured across n spatial units
    w               : W
                      spatial weights instance
    transformation  : string
                      weights transformation,  default is row-standardized "r".
                      Other options include "B": binary,  "D":
                      doubly-standardized,  "U": untransformed
                      (general weights), "V": variance-stabilizing.
    Attributes
    ----------
    y            : array
                   original variable
    w            : W
                   original w object
    I            : float
                   value of Moran's I
    """


    def __init__(self, y, w, transformation="r"):
        #Y would become either origins or destination locations (x,y)   
        self.y = y
        w.transform = transformation
        self.w = w #W is the definition of neighbors 
        self.__moments()
        self.I = self.__calc(self.z)
        
                
    #Calculates moments (first and second) needs to be updated to match math in the paper                
    def __moments(self):
        self.n = len(self.y)
        y = self.y
        z = y - y.mean()
        self.z = z
        self.z2ss = sum(z * z)
        self.EI = -1. / (self.n - 1)
        s1 = self.w.s1
        s0 = self.w.s0
        s2 = self.w.s2
        s02 = s0 * s0

    def __calc(self, z):
        zl = slag(self.w, z) # calculate weighted attribute - look into slag and see whats going on
        inum = sum(z * zl) #numerator of I, sum of weights times attribute at one location times attribute at another loc
        return self.n / self.w.s0 * inum / self.z2ss #w.s0 is the sum of the weights, z2ss is the sum of squares of Zi

def slag(w, y):
    """
    Spatial lag operator. 
    If w is row standardized, returns the average of each observation's neighbors; 
    if not, returns the weighted sum of each observation's neighbors.
    Parameters
    ----------
    w                   : W 
                          object
    y                   : array
                          numpy array with dimensionality conforming to w (see examples)
    Returns
    -------
    wy                  : array
                          array of numeric values for the spatial lag
    """
    return w.sparse * y

In [4]:
import pysal
import numpy as np
w = pysal.open(pysal.examples.get_path("stl.gal")).read()
f = pysal.open(pysal.examples.get_path("stl_hom.txt"))
y = np.array(f.by_col['HR8893'])
mi = BasicMoran(y,  w)
print "%7.5f" % mi.I

0.24366


In [67]:
vecs_A = np.array([[1, 55, 60, 100, 500], [2, 60, 55, 105, 501], [3, 500, 55, 155, 500], [4, 505, 60, 160, 500], [5, 105, 950, 105, 500], [6, 155, 950, 155, 499]])

In [232]:
origins = np.array([vecs_A[:,1], vecs_A[:,2]]).transpose()

In [229]:
origins

array([[ 55,  60],
       [ 60,  55],
       [500,  55],
       [505,  60],
       [105, 950],
       [155, 950]])

In [223]:
vecs_A

array([[  1,  55,  60, 100, 500],
       [  2,  60,  55, 105, 501],
       [  3, 500,  55, 155, 500],
       [  4, 505,  60, 160, 500],
       [  5, 105, 950, 105, 500],
       [  6, 155, 950, 155, 499]])

In [230]:
dests = np.array([vecs_A[:,3], vecs_A[:,4]]).transpose()

In [231]:
dests

array([[100, 500],
       [105, 501],
       [155, 500],
       [160, 500],
       [105, 500],
       [155, 499]])

In [241]:
class VecMoran:
    """Moran's I Global Autocorrelation Statistic
    Parameters
    ----------
    y               : array
                      variable measured across n spatial units
    w               : W
                      spatial weights instance
    transformation  : string
                      weights transformation,  default is row-standardized "r".
                      Other options include "B": binary,  "D":
                      doubly-standardized,  "U": untransformed
                      (general weights), "V": variance-stabilizing.
    Attributes
    ----------
    y            : array
                   original variable
    w            : W
                   original w object
    I            : float
                   value of Moran's I
    """


    def __init__(self, origins, destinations, transformation="r"):
        #Y would become either origins or destination locations (x,y)   
        self.origins = origins
        self.dests = destinations
        
        self.n = len(origins)
        xObar = self.origins[:,0].mean()
        yObar = self.origins[:,1].mean()
        xDbar = self.dests[:,0].mean()
        yDbar = self.dests[:,1].mean()
        u = (y[:,3] - y[:,1]) - (xDbar - xObar)
        v = (y[:,4] - y[:,2]) - (yDbar - yObar)
        z = np.outer(u, u) + np.outer(v,v)
        wo = self.wO(vectors = y)
        wd = self.wD(vectors = y)
        
        self.IO = n / np.sum(wo) * np.sum(wo*z) / sum(u**2 +v**2)
        self.ID = n / np.sum(wd) * np.sum(wd*z) / sum(u**2 +v**2)
        
        #print IO, ID, u, v, z
    
    #def __moments(self):
    #    zO = self.dests - self.dests.mean()
    #    zD = self.origins - self.origins.mean()
    #    self.zO = zO
    #    self.zD = zD
    #    self.zO2ss = sum(zO * zO)
    #    self.zD2ss = sum(
    #    self.EI = -1. / (self.n - 1)
    #    s1 = self.w.s1
    #    s0 = self.w.s0
    #    s2 = self.w.s2
    #    s02 = s0 * s0

        
    def wO(self, vectors, beta = -1.5): 
        if vectors == None:
            vectors = self.y
        origin_W = dist.squareform(dist.pdist(vectors[:,1:3])) ** (beta)
        np.fill_diagonal(origin_W, 0)
        return origin_W
    
    def wD(self, vectors, beta = -1.5):
        dest_W = dist.squareform(dist.pdist(vectors[:,3:5])) ** (beta)
        np.fill_diagonal(dest_W, 0)
        return dest_W
        

In [234]:
V = VecMoran(origins, dests)

/home/ljw/dev/ipython/IPython/kernel/__main__.py:60: RuntimeWarning: divide by zero encountered in power
/home/ljw/dev/ipython/IPython/kernel/__main__.py:65: RuntimeWarning: divide by zero encountered in power


In [237]:
import pysal as ps

In [236]:
V.IO

0.64594459436702134

In [212]:
vecs_A[:,1].mean()

230.0

In [95]:
V= VecMoran(vecs_A)

/home/ljw/dev/ipython/IPython/kernel/__main__.py:2: RuntimeWarning: divide by zero encountered in power
  from IPython.kernel.zmq import kernelapp as app
/home/ljw/dev/ipython/IPython/kernel/__main__.py:40: RuntimeWarning: invalid value encountered in double_scalars
/home/ljw/dev/ipython/IPython/kernel/__main__.py:41: RuntimeWarning: invalid value encountered in double_scalars


In [99]:
V.IO

nan

In [72]:
q = wO(vecs_A)

/home/ljw/dev/ipython/IPython/kernel/__main__.py:2: RuntimeWarning: divide by zero encountered in power
  from IPython.kernel.zmq import kernelapp as app


In [74]:
y = vecs_A

In [77]:
n = len(y)
xDbar = np.sum(y[:,1])/n
xObar = np.sum(y[:,2])/n
yDbar = np.sum(y[:,3])/n
yObar = np.sum(y[:,4])/n

In [78]:
u = (y[:,3] - y[:,1]) - (xDbar - xObar)
v = (y[:,4] - y[:,2]) - (yDbar - yObar)
z = np.outer(u, u) + np.outer(v,v)

In [81]:
np.dot(q, z)

array([[ inf,  inf,  inf,  inf, -inf, -inf],
       [ inf,  inf,  inf,  inf, -inf, -inf],
       [ inf,  inf,  inf,  inf, -inf, -inf],
       [ inf,  inf,  inf,  inf, -inf, -inf],
       [-inf, -inf, -inf, -inf,  inf,  inf],
       [-inf, -inf, -inf, -inf,  inf,  inf]])

In [32]:
print VecMoran(vecs_A)

1.43511974822 nan [ 170  170 -220 -220  125  125] [810 815 815 810 -80 -80] [[685000 689050 622750 618700 -43550 -43550]
 [689050 693125 626825 622750 -43950 -43950]
 [622750 626825 712625 708550 -92700 -92700]
 [618700 622750 708550 704500 -92300 -92300]
 [-43550 -43950 -92700 -92300  22025  22025]
 [-43550 -43950 -92700 -92300  22025  22025]]
<__main__.VecMoran instance at 0x7febca67f4d0>


/home/ljw/dev/ipython/IPython/kernel/__main__.py:6: RuntimeWarning: divide by zero encountered in double_scalars
/home/ljw/dev/ipython/IPython/kernel/__main__.py:40: RuntimeWarning: invalid value encountered in double_scalars


In [20]:
-1.0/6-1.0

-1.1666666666666667

In [66]:
vecs_A

array([[  1,  55,  60, 100, 500],
       [  2,  60,  55, 105, 500],
       [  3, 500,  55, 155, 500],
       [  4, 505,  60, 160, 500],
       [  5, 105, 950, 105, 500],
       [  6, 155, 950, 155, 500]])

In [68]:
def wO(vectors): 
    origin_W = dist.squareform(dist.pdist(vectors[:,1:3])) ** (-1.5)
    d0 = np.diag(

In [87]:
np.ones((6,6)) * np.diag([1]*len(vecs_A))

array([[ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.]])

In [71]:
print wO(vecs_A)

[[             inf   5.31829590e-02   1.06516985e-04   1.04756560e-04
    3.75740994e-05   3.73102890e-05]
 [  5.31829590e-02              inf   1.08348022e-04   1.06516985e-04
    3.72771808e-05   3.70353225e-05]
 [  1.06516985e-04   1.08348022e-04              inf   5.31829590e-02
    3.26812779e-05   3.36621172e-05]
 [  1.04756560e-04   1.06516985e-04   5.31829590e-02              inf
    3.28086285e-05   3.38124470e-05]
 [  3.75740994e-05   3.72771808e-05   3.26812779e-05   3.28086285e-05
               inf   2.82842712e-03]
 [  3.73102890e-05   3.70353225e-05   3.36621172e-05   3.38124470e-05
    2.82842712e-03              inf]]


/home/ljw/dev/ipython/IPython/kernel/__main__.py:2: RuntimeWarning: divide by zero encountered in power
  from IPython.kernel.zmq import kernelapp as app


In [69]:
def wD(vectors):
    return dist.squareform(dist.pdist(vectors[:,3:5])) ** (-1.5)

In [70]:
print wD(vecs_A)

[[        inf  0.08685003  0.00245164  0.00215166  0.08944272  0.00245103]
 [ 0.08685003         inf  0.00282758  0.00245103  1.          0.00282504]
 [ 0.00245164  0.00282758         inf  0.08944272  0.00282843  1.        ]
 [ 0.00215166  0.00245103  0.08944272         inf  0.00245164  0.08685003]
 [ 0.08944272  1.          0.00282843  0.00245164         inf  0.00282758]
 [ 0.00245103  0.00282504  1.          0.08685003  0.00282758         inf]]


/home/ljw/dev/ipython/IPython/kernel/__main__.py:2: RuntimeWarning: divide by zero encountered in power
  from IPython.kernel.zmq import kernelapp as app


In [50]:
dist.squareform(dist.pdist(vecs_A[:, 3:5]))

array([[  0.,   5.,  55.,  60.,   5.,  55.],
       [  5.,   0.,  50.,  55.,   0.,  50.],
       [ 55.,  50.,   0.,   5.,  50.,   0.],
       [ 60.,  55.,   5.,   0.,  55.,   5.],
       [  5.,   0.,  50.,  55.,   0.,  50.],
       [ 55.,  50.,   0.,   5.,  50.,   0.]])

In [59]:
vecs_A.shape

(6, 5)